# LGBM for the emergency demand predictions

https://www.kaggle.com/robikscube/tutorial-time-series-forecasting-with-xgboost

Importing libraries

In [ ]:
# general routine set up:

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# get wd and change the wd plus import libraries
import os
os.getcwd()
#os.chdir("/Users/Manu/Dropbox/CBS MSc Thesis Research Folder/DATA & Code/Model Specific Notebooks")


# standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from math import sqrt
import pickle

# ML
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
import tensorflow.keras.backend as K
from lightgbm import LGBMRegressor

Loading the data

In [ ]:
# this allows for accessing files stored in your google drive using the path "/gdrive/"
# mounting google drive locally:

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#loading the hourly traffic data (1 year of data; June 2018 to June 2019)
filename = "/gdrive/My Drive/Colab Notebooks/emergency_dispatches_bronx_H"
infile = open(filename,'rb')
emergency_ts = pickle.load(infile)
infile.close()

Preprocessing: already done


Time Series data must be re-framed as a supervised learning dataset before we can start using machine learning algorithms.  
There is no concept of input and output features in time series. Instead, we must choose the variable to be predicted and use feature engineering to construct all of the inputs that will be used to make predictions for future time steps.

## Feature Engineering for Times Series

[feature_eng_ts.png](attachment:feature_eng_ts.png)

In this tutorial, we will look at three classes of features that we can create from our time series dataset:

    Date Time Features: these are components of the time step itself for each observation.
    Lag Features: these are values at prior time steps.
    Window Features: these are a summary of values over a fixed window of prior time steps.


Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

The goal of feature engineering is to provide strong and ideally simple relationships between new input features and the output feature for the supervised learning algorithm to model.

### Preprocessing for monday the 16

In [ ]:
## Remove last 15 days of data since they are either erroneous or part of the test set to the robustness checks
emergency_ts_mon = emergency_ts.iloc[0:-360]
emergency_ts_mon.shape

(8400,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_mon[0:-n_test]
test = emergency_ts_mon[-n_test:]
print(train.shape)
print(test.shape)

(8376,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8293, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8293, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


$\textbf{General setup of the algorithm}:$ As far as general parameters go, the booster "gbtree" has been used here, ie. I have been using tree based models in each iteration instead of a linear model, which is rarely used. I have started out with a number of estimators of 1,000 and then I have fine tuned the following parameters after try 2: max_depth and min_child_weight (gsearch1), reg_alpha (gsearch2), gamma (gsearch3), subsample and colsample_bytree (gsearch4), reg_lambda (gsearch5).  
I have used a feature importance plot of the XGBClassfier model to select the most important features to include. I have included the top 10 features out of the 24 features provided from try 3 onwards.
The fine tuned parameters have the following influence on the xg boosting algorithm:  
  
$\textit{max_depth:}$ specifies the max depth of a tree and can be used to control overfitting as higher depth will allow the model to learn relations very specific to a particular sample   
$\textit{min_child_weight:}$ this sets the minimum sum of weights of all observations required in a child. Higher values prevent the model from learning too specific relations.  
$\textit{reg_alpha:}$ L1 regularization term. Can be used in case of high dimensionality to make the algorithm run faster. Can be a solution to overfitting in case of a relatively small dataset.  
$\textit{gamma:}$ sets the minimum loss function required to make a split.  
$\textit{subsample:}$ sets the fraction of observations to be random samples of each tree. lower values prevent overfitting but small too small values might lead to underfitting.  
$\textit{colsample_bytree:}$ fraction of columns to be random samples of each tree.  
$\textit{reg_lambda:}$ L2 regularization term. can be a solution to overfitting in case of a relatively small dataset. Can be explored to reduce overfitting.

#LightGBM (Gradient Boosting)



https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html

lightgbm parameters:

https://lightgbm.readthedocs.io/en/latest/Parameters.html


optimal grid (August 25) :
{'estimator__colsample_bytree': 0.8,
 'estimator__learning_rate': 0.1,
 'estimator__max_depth': 8,
 'estimator__min_child_samples': 30,
 'estimator__min_child_weight': 0.01,
 'estimator__n_estimators': 250,
 'estimator__num_leaves': 30,
 'estimator__reg_alpha': 0.0,
 'estimator__reg_lambda': 0.7,
 'estimator__subsample': 1.0}

## Fitting the optimal model

In [ ]:
# training the best model (24 steps ahead)
lgbm_reg_opt_new = LGBMRegressor(random_state=1,learning_rate=0.01, objective = "root_mean_squared_error", n_estimators =250, colsample_bytree = 0.8, max_depth = 15, subsample= 1.0, num_leaves=150, min_child_samples=20, min_child_weight =3, reg_lambda = 0.0, reg_alpha= 0.0, eval_metric = "rmse",boosting_type="gbdt")

lgbm_moreg_fit_new = MultiOutputRegressor(lgbm_reg_opt_new).fit(X_train, Y_train)

## Predictions for monday the 16

In [ ]:
# 24 steps ahead predictions
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test)

In [ ]:
# MSE
mse_lgbm_new = mean_squared_error(Y_test, lgbm_preds_new)
sqrt(mse_lgbm_new)

## Predictions for tuesday the 17

### Preprocessing for tuesday the 17

In [ ]:
##
emergency_ts_tue = emergency_ts.iloc[0:-360 + 24]
emergency_ts_tue.shape

(8424,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_tue[0:-n_test]
test = emergency_ts_tue[-n_test:]
print(train.shape)
print(test.shape)

(8400,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8317, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8317, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# 24 steps ahead predictions
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test)

In [ ]:
# RMSE
mse_lgbm_new = mean_squared_error(Y_test, lgbm_preds_new)
sqrt(mse_lgbm_new)

5.701809247539146

## Predictions for wedensday the 18

### Preprocessing for wedensday the 18

In [ ]:
##
emergency_ts_wed = emergency_ts.iloc[0:-360 + 24*2]
emergency_ts_wed.shape

(8448,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_wed[0:-n_test]
test = emergency_ts_wed[-n_test:]
print(train.shape)
print(test.shape)

(8424,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8341, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8341, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# 24 steps ahead predictions
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test)

In [ ]:
# RMSE
mse_lgbm_new = mean_squared_error(Y_test, lgbm_preds_new)
sqrt(mse_lgbm_new)

6.289150620708447

## Predictions for thursday the 19

### Preprocessing for thursday the 19

In [ ]:
##
emergency_ts_thu = emergency_ts.iloc[0:-360 + 24*3]
emergency_ts_thu.shape

(8472,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_thu[0:-n_test]
test = emergency_ts_thu[-n_test:]
print(train.shape)
print(test.shape)

(8448,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8365, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8365, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# 24 steps ahead predictions
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test)

In [ ]:
# RMSE
mse_lgbm_new = mean_squared_error(Y_test, lgbm_preds_new)
sqrt(mse_lgbm_new)

5.430993191142919

## Predictions for friday the 20

### Preprocessing for friday the 20

In [ ]:
##
emergency_ts_fri = emergency_ts.iloc[0:-360 + 24*4]
emergency_ts_fri.shape

(8496,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_fri[0:-n_test]
test = emergency_ts_fri[-n_test:]
print(train.shape)
print(test.shape)

(8472,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8389, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8389, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# 24 steps ahead predictions
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test)

In [ ]:
# RMSE
mse_lgbm_new = mean_squared_error(Y_test, lgbm_preds_new)
sqrt(mse_lgbm_new)

5.645653897366603

## Predictions for saturday the 21

### Preprocessing for saturday the 21

In [ ]:
##
emergency_ts_sat = emergency_ts.iloc[0:-360 + 24*5]
emergency_ts_sat.shape

(8520,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_sat[0:-n_test]
test = emergency_ts_sat[-n_test:]
print(train.shape)
print(test.shape)

(8496,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8413, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8413, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# 24 steps ahead predictions
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test)

In [ ]:
# RMSE
mse_lgbm_new = mean_squared_error(Y_test, lgbm_preds_new)
sqrt(mse_lgbm_new)

6.660966467734799

## Predictions for sunday the 22

### Preprocessing for sunday the 22

In [ ]:
##
emergency_ts_sun = emergency_ts.iloc[0:-360 + 24*6]
emergency_ts_sun.shape

(8544,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_sun[0:-n_test]
test = emergency_ts_sun[-n_test:]
print(train.shape)
print(test.shape)

(8520,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8437, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8437, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# 24 steps ahead predictions
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test)

In [ ]:
# RMSE
mse_lgbm_new = mean_squared_error(Y_test, lgbm_preds_new)
sqrt(mse_lgbm_new)

4.489210961067223